In [ ]:
import imutils
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from skimage import data, filters
from skimage.filters import gaussian
from skimage.segmentation import active_contour
# plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
# Function to read image saved on google drive
def readImage(number, jpg = False):
    if jpg == True:
        path = "images/" + str(number) + ".jpg"
    else:
        path = "images/" + str(number) + ".tif"
    img = cv.imread(path)
    return img[:,:,::-1]

# Function to display image using matplotlib
def displayImage(a, title1 = "Img"):
    plt.imshow(a, cmap='gray', vmin=0, vmax=255), plt.title(title1)
    plt.show()

In [ ]:
# Area-based cleaning of image
def clean(img, areaThreshold = 25):
    # Find all connected components
    output = cv.connectedComponentsWithStats(np.uint8(img), 4, cv.CV_32S)
    num_labels = output[0]
    labels = output[1]
    stats = output[2]
    centroids = output[3]

    # output image
    cleaned = np.copy(np.uint8(img))
    # if area of any connected component is less than area threshold, remove that component
    for i in range(cleaned.shape[0]):
        for j in range(cleaned.shape[1]):
            if stats[labels[i][j], cv.CC_STAT_AREA] < areaThreshold:
                cleaned[i][j] = 0
    return cleaned

In [ ]:
# Extract green channel from image
def extractChannel(img, channel = 'g'):
    # RGB instead of BGR bcooz we flipped it earlier while reading image
    r, g, b = cv.split(img)
    if channel == 'g':
        return g
    if channel == 'r':
        return r
    return b

In [ ]:
# Function to apply clahe on input image
def applyCLAHE(img):
    # hyper-parameters in clahe
    limit=3; grid=(7,7)
    clahe = cv.createCLAHE(clipLimit=limit, tileGridSize=grid)
    # output image
    img = clahe.apply(img)
    return img

In [ ]:
def diskKernel(n):
    return cv.getStructuringElement(cv.MORPH_ELLIPSE,(2*n - 1 , 2*n - 1))

### Finding centre

In [ ]:
img = readImage(1)
displayImage(img)
b = extractChannel(img, 'b')
cl = applyCLAHE(b)
filtered = cv.medianBlur(cl,5)
opening = cv.morphologyEx(filtered, cv.MORPH_OPEN, diskKernel(5))
ret,mask = cv.threshold(opening, 127, 255,cv.THRESH_BINARY)
displayImage(mask, 'mask')

eroding = cv.erode(mask, diskKernel(3) , iterations = 7)
dilate = cv.dilate(eroding, diskKernel(3) , iterations = 15)
close = cv.morphologyEx(dilate, cv.MORPH_CLOSE, diskKernel(5) , iterations = 14)
displayImage(close)

cnts = cv.findContours(close.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
for c in cnts:
    # compute the center of the contour
    M = cv.moments(c)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    print(str(cX) + " " + str(cY))

### Finding contour

In [ ]:
s = np.linspace(0, 2*np.pi, 400)
r = cY + 50*np.sin(s)
c = cX + 50*np.cos(s)
init = np.array([r, c]).T
g = extractChannel(img)
snake = active_contour(gaussian(g, 3),
                       init, alpha=0.01, beta=1, gamma=0.0001,
                       coordinates='rc')

fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(g, cmap=plt.cm.gray)
ax.plot(init[:, 1], init[:, 0], '--r', lw=3)
ax.plot(snake[:, 1], snake[:, 0], '-b', lw=3)
ax.set_xticks([]), ax.set_yticks([])
ax.axis([0, g.shape[1], g.shape[0], 0])

plt.show()

### Extract mask

In [ ]:
snake = np.array(snake, dtype=np.float32)
final_mask = np.empty(img.shape)
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        final_mask[i,j] = cv.pointPolygonTest(snake, (i, j), True)
displayImage(final_mask)

# Bring pixel values to [0, 1]
final_mask -= np.min(final_mask)
final_mask /= np.max(final_mask)
print(np.min(final_mask), np.max(final_mask))
final_mask[final_mask >= 0.95] = 1
final_mask[final_mask < 0.95] = 0

# closing
close = cv.morphologyEx(final_mask, cv.MORPH_CLOSE, diskKernel(5))
plt.imsave('output_part3/1.png', close, cmap='gray')